# FLiES-ANN Sensitivity Analysis with ECOv002 Cal-Val

This notebook performs a sensitivity analysis of the FLiES-ANN model using ECOSTRESS Collection 2 Cal-Val data. It loads input data, processes it through the FLiES-ANN model, and visualizes the impact of input perturbations on latent heat flux. The notebook also generates figures for publication and explores the relationship between surface temperature and latent heat flux.

## Import Required Libraries and Functions

This cell imports all necessary libraries and functions for data processing, model execution, statistical analysis, and plotting. It includes custom modules for the BESS-JPL model, sensitivity analysis, and net radiation calculations, as well as standard scientific Python libraries.

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

In [2]:
from typing import Callable
from os import makedirs
from os.path import join
import numpy as np
import pandas as pd
from FLiESANN import process_FLiESANN_table, load_ECOv002_calval_FLiESANN_inputs
from monte_carlo_sensitivity import perturbed_run, sensitivity_analysis, divide_absolute_by_unperturbed
import matplotlib.pyplot as plt
from scipy.stats import mstats
import seaborn as sns
from matplotlib.ticker import FuncFormatter

[2025-10-28 09:23:47 INFO] SRTM working directory: ~/data/NASADEM
[2025-10-28 09:23:47 INFO] SRTM download directory: ~/data/NASADEM
[2025-10-28 09:23:47 INFO] SRTM download directory: ~/data/NASADEM


## Set Normalization Function

This cell assigns the normalization function used to compare perturbed model outputs to the unperturbed baseline. The function `divide_absolute_by_unperturbed` is used for normalization in the sensitivity analysis.

In [3]:
normalization_function = divide_absolute_by_unperturbed

## Load and Filter Input Data

This cell loads the ECOSTRESS Cal-Val input data using a custom loader function. The resulting DataFrame is displayed for inspection.

In [4]:
input_df = load_ECOv002_calval_FLiESANN_inputs()
input_df

,Unnamed: 0.1,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,...,NIR_diffuse_Wm2,PAR_direct_Wm2,NIR_direct_Wm2,atmospheric_transmittance,UV_proportion,PAR_proportion,NIR_proportion,UV_diffuse_fraction,PAR_diffuse_fraction,NIR_diffuse_fraction
0,0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,...,[0.],[290.52757544],[370.12899236],0.828330,0.061078,0.435810,0.505289,0.428838,0.089926,0.000000
1,1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,...,[0.],[437.97749268],[535.43787771],0.877691,0.060632,0.432951,0.508679,0.293250,0.038945,0.000000
2,2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,...,[0.],[441.86860931],[539.03713748],0.879383,0.060702,0.432891,0.508672,0.291212,0.036763,0.000000
3,3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,...,[0.],[401.12769355],[500.61248237],0.859748,0.059537,0.433195,0.509777,0.305537,0.057074,0.000000
4,4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,...,[4.71651513],[347.78380538],[439.98515647],0.839070,0.059503,0.434472,0.508497,0.345358,0.084691,0.010606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,...,[8.18987992],[138.44108644],[189.18666313],0.761285,0.048997,0.429992,0.524415,0.522248,0.144571,0.041494
1061,1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,...,[7.0025377],[153.20291257],[204.56529318],0.773823,0.050539,0.433826,0.518544,0.514046,0.134461,0.033098
1062,1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,...,[5.03350805],[173.97592496],[229.46875606],0.789096,0.051973,0.435491,0.515249,0.497416,0.122232,0.021465
1063,1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,...,[0.],[217.94150599],[287.02839783],0.807714,0.054817,0.434206,0.513784,0.461087,0.101538,0.000000


In [5]:
list(input_df.columns)

['Unnamed: 0.1',
 'Unnamed: 0',
 'ID',
 'vegetation',
 'climate',
 'STICinst',
 'BESSinst',
 'MOD16inst',
 'PTJPLSMinst',
 'ETinst',
 'ETinstUncertainty',
 'PET',
 'Rn',
 'ESI',
 'RH',
 'Ta',
 'LST',
 'SM',
 'NDVI',
 'NDVI-UQ',
 'albedo',
 'albedo-UQ',
 'LST_err',
 'view_zenith',
 'Rg',
 'EmisWB',
 'time_utc',
 'solar_time',
 'solar_hour',
 'local_time',
 'LE',
 'LE_filt',
 'LEcorr25',
 'LEcorr50',
 'LEcorr75',
 'LEcorr_ann',
 'H_filt',
 'Hcorr25',
 'Hcorr50',
 'Hcorr75',
 'Hcorr_ann',
 'NETRAD_filt',
 'G_filt',
 'SM_surf',
 'SM_rz',
 'AirTempC',
 'SW_IN',
 'RH_percentage',
 'ESIrn_STIC',
 'ESIrn_PTJPLSM',
 'ESIrn_MOD16',
 'ESIrn_BESS',
 'ESIrn_Unc_ECO',
 'ESIrn_LEcorr50',
 'JET',
 'eco_time_utc',
 'Site Name',
 'Date-Time',
 'Site ID',
 'Name',
 'Lat',
 'Long',
 'elevation_m',
 'Clim',
 'Veg',
 'MAT',
 'MAP',
 'StartDate',
 'EndDate',
 'LE_count',
 'closure_ratio',
 'geometry',
 'time_UTC',
 'ST_K',
 'ST_C',
 'Ta_C',
 'SWin_Wm2',
 'emissivity',
 'insitu_Rn_daylight_Wm2',
 'insitu_LE_d

## Process Input Data Through Model

This cell applies the processing function to the filtered input data, running it through the Verma net radiation and PT-JPL-SM model, and displays the resulting DataFrame.

In [6]:
processed = process_FLiESANN_table(input_df)
processed

[2025-10-28 09:23:48 INFO] started processing FLiES input table
[2025-10-28 09:23:48 INFO] processing table in vectorized mode
[2025-10-28 09:23:48 INFO] processing 1065 rows in vectorized mode
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[2025-10-28 09:24:00 INFO] completed processing FLiES input table


,Unnamed: 0.1,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,...,NIR_diffuse_Wm2,PAR_direct_Wm2,NIR_direct_Wm2,atmospheric_transmittance,UV_proportion,PAR_proportion,NIR_proportion,UV_diffuse_fraction,PAR_diffuse_fraction,NIR_diffuse_fraction
0,0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,...,0.000000,290.492175,370.061355,0.828220,0.061075,0.435837,0.505264,0.428990,0.089973,0.000000
1,1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,...,0.000000,437.988529,535.473997,0.877727,0.060632,0.432938,0.508693,0.293193,0.038931,0.000000
2,2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,...,0.000000,441.879624,539.073520,0.879420,0.060702,0.432878,0.508685,0.291155,0.036749,0.000000
3,3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,...,0.000000,401.138203,500.646568,0.859784,0.059538,0.433182,0.509790,0.305480,0.057060,0.000000
4,4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,...,4.714074,347.796380,440.019885,0.839109,0.059503,0.434459,0.508510,0.345300,0.084674,0.010600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,...,8.190341,138.437171,189.167960,0.761245,0.048996,0.430012,0.524395,0.522320,0.144589,0.041500
1061,1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,...,7.003365,153.199642,204.545828,0.773786,0.050539,0.433847,0.518522,0.514120,0.134479,0.033105
1062,1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,...,5.034356,173.970257,229.450125,0.789060,0.051972,0.435507,0.515233,0.497488,0.122254,0.021470
1063,1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,...,0.000000,217.930879,287.005659,0.807671,0.054816,0.434220,0.513771,0.461162,0.101562,0.000000


## Accuracy Comparison of In-Situ Solar Radiation to FLiES-ANN Solar Radiation

In [7]:
sc = plt.scatter(
    x=processed.insitu_SWin_Wm2,
    y=processed.SWin_Wm2,
    # c=processed.ST_C,
    cmap="jet",
    color=None,
    s=5
)

# Add 1-to-1 line
min_val = min(processed.insitu_SWin_Wm2.min(), processed.SWin_Wm2.min())
max_val = max(processed.insitu_SWin_Wm2.max(), processed.SWin_Wm2.max())
plt.plot([min_val, max_val], [min_val, max_val], color='black', linestyle='-', linewidth=1, label='1:1 Line')

# Filter out NaN or infinite values
valid_data = processed.dropna(subset=["insitu_SWin_Wm2", "SWin_Wm2"])
valid_data = valid_data[np.isfinite(valid_data["insitu_SWin_Wm2"]) & np.isfinite(valid_data["SWin_Wm2"])]

# Add OLS regression line
if len(valid_data) > 1:  # Ensure there are enough points for regression
    coefs = np.polyfit(valid_data.insitu_SWin_Wm2, valid_data.SWin_Wm2, 1)
    ols_x = np.array([min_val, max_val])
    ols_y = coefs[0] * ols_x + coefs[1]
    plt.plot(ols_x, ols_y, color='black', linestyle='--', linewidth=1, label='OLS Fit')

    # Calculate R-squared and RMSE
    from sklearn.metrics import r2_score, mean_squared_error
    y_true = valid_data.SWin_Wm2
    y_pred = coefs[0] * valid_data.insitu_SWin_Wm2 + coefs[1]
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Annotate R-squared and RMSE on plot
    plt.annotate(f"$R^2$ = {r2:.2f}\nRMSE = {rmse:.1f}",
                 xy=(0.05, 0.95), xycoords='axes fraction',
                 ha='left', va='top',
                 fontsize=10,
                 bbox=dict(boxstyle='round', fc='white', ec='gray', alpha=0.7))
else:
    print("Not enough valid data points for regression.")

# Set ticks every 100 for both axes
xticks = np.arange(100 * (min_val // 100), 100 * (max_val // 100 + 2), 100)
yticks = np.arange(100 * (min_val // 100), 100 * (max_val // 100 + 2), 100)
plt.xticks(xticks)
plt.yticks(yticks)

plt.grid(True, zorder=0) 
plt.xlabel("In-Situ Solar Radiation (W/m$^2$)")
plt.ylabel("FLiES-ANN Solar Radiation (W/m$^2$)")
plt.title("Accuracy Comparison\nof In-Situ Solar Radiation\nto FLiES-ANN Solar Radiation")
# plt.colorbar(sc, label="ECOSTRESS Surface Temperature (°C)")
plt.legend()
plt.savefig("Unperturbed Comparison of In-Situ Solar Radiation to FLiES-ANN Solar Radiation.jpeg", format='jpeg', bbox_inches='tight')
plt.savefig("Unperturbed Comparison of In-Situ Solar Radiation to FLiES-ANN Solar Radiation.svg", format='svg', bbox_inches='tight')
plt.show()

AttributeError: 'DataFrame' object has no attribute 'insitu_SWin_Wm2'

## Plot Unperturbed Comparison of Albedo to Net Radiation

This cell creates a scatter plot comparing STARS albedo to FLiESANN solar radiation for the unperturbed data. The plot is saved as both JPEG and SVG files for publication or further analysis.

In [ ]:
plt.scatter(x=processed.albedo, y=processed.SWin_Wm2, color='black', s=10, zorder=5)
plt.grid(True, zorder=0)
plt.xlabel("ECOSTRESS Albedo")
plt.ylabel("BESS-JPL Net Radiation (W/m$^2$)")
plt.title("Unperturbed Comparison\nof ECOSTRESS Albedo\nto BESS-JPL Solar Radiation", pad=20)

plt.savefig("Unperturbed Comparison of ECOSTRESS Albedo to BESS-JPL Solar Radiation.jpeg", format='jpeg', bbox_inches='tight')
plt.savefig("Unperturbed Comparison of ECOSTRESS Albedo to BESS-JPL Solar Radiation.svg", format='svg', bbox_inches='tight')

plt.show()

## Further Filter Input Data and Check Temperature Range

This cell applies additional filters to the input data, ensuring only valid `fAPARmax` and `NDVI` values are included. It also checks the minimum and maximum surface temperature values in the filtered dataset.

In [ ]:
# input_df = pd.read_csv(input_filename)
input_df = load_ECOv002_calval_FLiESANN_inputs()

if "Ta" in input_df and "Ta_C" not in input_df:
    # input_df.rename({"Ta": "Ta_C"}, inplace=True)
    input_df["Ta_C"] = input_df["Ta"]

input_df = input_df[input_df.NDVI.apply(lambda NDVI: NDVI > 0.05)]

np.nanmin(input_df.albedo), np.nanmax(input_df.albedo)

## Check Number of Valid Input Rows

This cell displays the number of rows remaining in the input DataFrame after all filtering steps, confirming the size of the dataset used for analysis.

In [ ]:
len(input_df)

## Run Perturbed Model Analysis

This cell sets up the input and output variables for the sensitivity analysis and runs the `perturbed_run` function, which perturbs the input variable and observes the effect on the output variable using the BESS-JPL model. The results are displayed for further analysis.

In [ ]:
input_variable = "albedo"
output_variable = "SWin_Wm2"

Rn_results = perturbed_run(
    input_df=input_df, 
    input_variable=input_variable, 
    output_variable=output_variable, 
    forward_process=process_FLiESANN_table,
    normalization_function=normalization_function
)

Rn_results

In [ ]:
input_variable = "albedo"
output_variable = "SWin_Wm2"

SWin_results = perturbed_run(
    input_df=input_df, 
    input_variable=input_variable, 
    output_variable=output_variable, 
    forward_process=process_FLiESANN_table,
    normalization_function=normalization_function
)

SWin_results

## Filter Out NaN Results

This cell removes any rows with missing values from the perturbed results to ensure only valid data points are used in subsequent analysis and plotting.

In [ ]:
filtered_SWin_results = SWin_results.dropna()
filtered_SWin_results

## Plot Change in Albedo vs. Solar Radiation

This cell generates a scatter plot showing the relationship between changes in STARS albedo and changes in FLiES-ANN solar radiation due to input perturbations. The plot is saved as JPEG and SVG files.

In [ ]:
# plt.xticks(range(int(min(processed.ST_C)), int(max(processed.ST_C)) + 1, 5))
# plt.ylim(-350, 150)
plt.scatter(x=Rn_results.input_perturbation, y=Rn_results.output_perturbation, color='black', s=10, zorder=5)  # Adjust the 's' parameter to make dots thinner
plt.grid(True, zorder=0)
plt.xlabel("Change in STARS Albedo")
plt.ylabel("Change in FLiES-ANN Solar Radiation (W/m$^2$)")
plt.title("Comparison of Change\nin STARS Albedo\nto FLiES-ANN Solar Radiation", pad=20)

plt.savefig("Comparison of Change in STARS Albedo to FLiES-ANN Solar Radiation.jpeg", format='jpeg', bbox_inches='tight')
plt.savefig("Comparison of Change in STARS Albedo to FLiES-ANN Solar Radiation.svg", format='svg', bbox_inches='tight')

plt.show()

In [ ]:
sc = plt.scatter(
    x=Rn_results.input_perturbation, 
    y=Rn_results.output_perturbation, 
    c=Rn_results.input_unperturbed, 
    color=None,
    cmap="jet",
    s=3,
    alpha=0.5, 
    zorder=5
)

plt.grid(True, zorder=0)
plt.xlabel("Change in STARS Albedo")
plt.ylabel("Change in FLiES-ANN Solar Radiation (W/m$^2$)")
plt.title("Comparison of Change\nin STARS Albedo\nto FLiES-ANN Solar Radiation", pad=20)
plt.colorbar(sc, label="STARS Albedo")
plt.savefig("Comparison of Change in STARS Albedo to FLiES-ANN Solar Radiation.jpeg", format='jpeg', bbox_inches='tight')
plt.savefig("Comparison of Change in STARS Albedo to FLiES-ANN Solar Radiation.svg", format='svg', bbox_inches='tight')

plt.show()

## Compute Correlation Between Input and Output Perturbations

This cell calculates the Pearson correlation coefficient between the standardized input and output perturbations, quantifying the strength of their linear relationship.

In [ ]:
correlation = mstats.pearsonr(
    np.array(filtered_SWin_results.input_perturbation_std).astype(np.float64), 
    np.array(filtered_SWin_results.output_perturbation_std).astype(np.float64)
)[0]

correlation

## Run Full Sensitivity Analysis for Multiple Inputs

This cell performs a comprehensive sensitivity analysis by perturbing several input variables (surface temperature, NDVI, albedo, air temperature, relative humidity) and measuring their effect on latent heat flux. The results are summarized in a DataFrame.

In [ ]:
input_variables = ["albedo"]
output_variables = ["SWin_Wm2"]

Rn_perturbation_df, Rn_sensitivity_metrics_df = sensitivity_analysis(
    input_df=input_df,
    input_variables=input_variables,
    output_variables=output_variables,
    forward_process=process_FLiESANN_table,
    normalization_function=normalization_function
)

Rn_sensitivity_metrics_df

## Plot Sensitivity Magnitude Bar Chart

This cell creates a bar chart showing the average percent change in latent heat flux for each input variable, visualizing the magnitude of model sensitivity to each input. The plot is saved as JPEG and SVG files with the BESS-JPL label.

In [ ]:
# df = Rn_sensitivity_metrics_df
# df = df[(df.output_variable == "SWin_Wm2") & (df.metric == "mean_normalized_change")]
# ax = sns.barplot(x=df.input_variable, y=df.value * 100, color='black')
# ax.set_xticklabels(["Surface\nTemperature", "Albedo", "Air\nTemperature", "Relative\nHumidity"])
# plt.xlabel("Input Variable")
# plt.ylabel("Average Percent Change in Output Perturbation")
# plt.title("BESS-JPL Net Radiation Sensitivity Magnitude")
# # plt.ylim(0, 160)  # Set y-axis range from 0 to 160
# plt.grid(axis='y', color='lightgray', linestyle='-', linewidth=0.5)  # Add light gray horizontal gridlines only

# # Add percent sign to y-axis tick labels
# ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{int(y)}%'))

# plt.savefig("BESS-JPL Net Radiation Sensitivity Magnitude.jpeg", format='jpeg', bbox_inches='tight')
# plt.savefig("BESS-JPL Net Radiation Sensitivity Magnitude.svg", format='svg', bbox_inches='tight')

# plt.show()

## Summary and Next Steps

This notebook demonstrated a full sensitivity analysis workflow for the BESS-JPL model using ECOSTRESS Cal-Val data. Key results include the identification of input variables with the greatest influence on latent heat flux. Next steps could include further exploration of model parameters, additional visualizations, or application to other datasets.